In [1]:
import numpy as np
import tensorflow as tf
import random
from time import time

%load_ext autoreload
%autoreload 2

In [2]:
from translate.model import Model

In [48]:
def self_test_model():
    """
    Test the translation model.
    """
    def linebreak():
        print('-' * 50)
    
    print("Self-test for neural translation model.")
    linebreak()
    
    graph = tf.Graph()
    
    with graph.as_default():
        with tf.device('/cpu:0'):
            t = time()
            # Create model with vocabularies of 10, 2 small buckets, 2 layers of 32.
            model = Model(source_vocab_size=10, 
                          target_vocab_size=10,
                          buckets=[(3, 3), (6, 6)], 
                          size=32,
                          num_layers=2,
                          learning_rate=None,
                          max_gradient_norm=5.0, 
                          batch_size=32,
                          use_lstm=True, 
                          use_local=True,
                          optim='adam',
                          num_samples=None)
            
            print("Initializing Model took %.6fs" %(time() - t))
            linebreak()
    
    with tf.Session(graph=graph) as sess:
        
        t = time()
        sess.run(tf.initialize_all_variables())
        print("Initializing Variables took %.6fs" %(time() - t))
        linebreak()

        # Fake data set for both the (3, 3) and (6, 6) bucket.
        data_set = ([([1, 1], [2, 2]), ([3, 3], [4]), ([5], [6])],
                    [([1, 1, 1, 1, 1], [2, 2, 2, 2, 2]), ([3, 3, 3], [5, 6])])
        num_iter = 20
        
        print('Using Learning Rate: %.2f' %(model.learning_rate.eval()))
        linebreak()
        
        t = time()
        # Train the fake model for 5 steps.
        for _ in xrange(num_iter):
            bucket_id = random.choice([0, 1])
            encoder_inputs, decoder_inputs, target_weights = model.get_batch(data_set, bucket_id)
            loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, False)
            print('Perplexity: %f' %(np.exp(loss)))
        linebreak()
        print("Average training time: %.6fs/iter" %((time() - t)/num_iter))

In [49]:
self_test_model()

Self-test for neural translation model.
--------------------------------------------------
Initializing Model took 8.957379s
--------------------------------------------------
Initializing Variables took 4.395649s
--------------------------------------------------
Using Learning Rate: 0.01
--------------------------------------------------
Perplexity: 5.452719
Perplexity: 3.408495
Perplexity: 2.830749
Perplexity: 3.462721
Perplexity: 1.738196
Perplexity: 4.457092
Perplexity: 3.844059
Perplexity: 2.823817
Perplexity: 1.789566
Perplexity: 1.629817
Perplexity: 1.521011
Perplexity: 1.398613
Perplexity: 1.226592
Perplexity: 1.213510
Perplexity: 2.402075
Perplexity: 2.415878
Perplexity: 1.075651
Perplexity: 1.052696
Perplexity: 1.694463
Perplexity: 1.062993
--------------------------------------------------
Average training time: 0.032712s/iter


In [52]:
def self_test_model_forward():
    """
    Test the translation model.
    """
    def linebreak():
        print('-' * 50)
    
    print("Self-test for neural translation model.")
    linebreak()
    
    graph = tf.Graph()
    
    with graph.as_default():
        with tf.device('/cpu:0'):
            t = time()
            # Create model with vocabularies of 10, 2 small buckets, 2 layers of 32.
            model = Model(source_vocab_size=10, 
                          target_vocab_size=10,
                          buckets=[(3, 3), (6, 6)], 
                          size=32,
                          num_layers=2,
                          learning_rate=None, 
                          batch_size=32, 
                          forward_only=True)
            
            print("Initializing Model took %.6fs" %(time() - t))
            linebreak()
    
    with tf.Session(graph=graph) as sess:
        
        t = time()
        sess.run(tf.initialize_all_variables())
        print("Initializing Variables took %.6fs" %(time() - t))
        linebreak()

        # Fake data set for both the (3, 3) and (6, 6) bucket.
        data_set = ([([1, 1], [2, 2]), ([3, 3], [4]), ([5], [6])],
                    [([1, 1, 1, 1, 1], [2, 2, 2, 2, 2]), ([3, 3, 3], [5, 6])])
        bucket_id = 1
        
        t = time()
        # Test the output of the fake model.
        encoder_inputs, decoder_inputs, target_weights = model.get_batch(data_set, bucket_id)
        output = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, True)
        print("Model runtime was %.6fs" %(time() - t))
        
        return output

In [53]:
output = self_test_model_forward()

Self-test for neural translation model.
--------------------------------------------------
Initializing Model took 1.762947s
--------------------------------------------------
Initializing Variables took 1.057347s
--------------------------------------------------
Model runtime was 0.047165s


In [3]:
from translate.translate import get_data
from translate.translate import _BUCKETS

In [4]:
DATADIR = '/Users/PragaashP/Desktop/f16/cs294-129/final-proj/datasets/dev/processed'
EN_PATH = DATADIR + '/newstest2013.ids50000.en'
FR_PATH = DATADIR + '/newstest2013.ids50000.fr'
newstest2013 = get_data(en_ids_path=EN_PATH, fr_ids_path=FR_PATH)

In [6]:
def get_weights(data):
    std = np.sqrt(np.asarray(map(len, data)))
    return std.cumsum()/std.sum()

In [16]:
def train_model(data, checkpoint_dir):
    """
    Test the translation model.
    """
    def linebreak():
        print('-' * 50)
    
    print("Train neural translation model.")
    linebreak()
    
    graph = tf.Graph()
    
    with graph.as_default():
        with tf.device('/cpu:0'):
            t = time()
            model = Model(source_vocab_size=50000, 
                          target_vocab_size=50000,
                          buckets=_BUCKETS, 
                          size=128,
                          num_layers=3,
                          max_gradient_norm=5.0, 
                          batch_size=64,
                          use_lstm=True,
                          optim='adam',
                          num_samples=None)
            
            print("Initializing Model took %.6fs" %(time() - t))
            linebreak()
    
    with tf.Session(graph=graph) as sess:
        
        t = time()
        model.load(sess, checkpoint_dir)
        print("Loading Model took %.6fs" %(time() - t))
        linebreak()

        num_iter = 1001
        checkpoint = 10
        intervals = get_weights(data)
        
        print('Using Learning Rate: %.2f' %(model.learning_rate.eval()))
        linebreak()
        
        t = time()
        batch_loss, prev_losses = 0.0, []
        for i in xrange(1, num_iter):
            bucket_id = np.abs(np.random.rand() - intervals).argmin()
            encoder_inputs, decoder_inputs, target_weights = model.get_batch(data, bucket_id)
            loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, False)
            batch_loss += (loss/checkpoint)
            if i % checkpoint == 0:
                if len(prev_losses) > 2 and batch_loss > max(prev_losses[-3:]):
                    sess.run(model.learning_rate_decay_op)
                prev_losses.append(batch_loss)
                print('Perplexity: %f' %(np.exp(batch_loss)))
                batch_loss = 0.0
                
        linebreak()
        print("Average training time: %.6fs/iter" %((time() - t)/num_iter))
        model.save(sess, checkpoint_dir)

In [17]:
CKPT_DIR = '/Users/PragaashP/Desktop/f16/cs294-129/final-proj/nmt/checkpoints'
train_model(newstest2013, CKPT_DIR)

Train neural translation model.
--------------------------------------------------
Initializing Model took 93.722282s
--------------------------------------------------
Loading Model took 42.494763s
--------------------------------------------------
Using Learning Rate: 0.01
--------------------------------------------------
Perplexity: 6459.561820
Perplexity: 1442.489431
Perplexity: 886.798324
Perplexity: 972.945839
Perplexity: 798.610501
Perplexity: 680.120646
Perplexity: 498.986414
Perplexity: 440.391930
Perplexity: 434.633080
Perplexity: 326.504664
Perplexity: 404.900369
Perplexity: 287.410460
Perplexity: 375.540878
Perplexity: 247.719276
Perplexity: 272.915217
Perplexity: 217.843840
Perplexity: 287.530368
Perplexity: 292.494319
Perplexity: 178.639094
Perplexity: 187.787048
Perplexity: 185.398113
Perplexity: 110.513220
Perplexity: 105.202938
Perplexity: 129.426412
Perplexity: 56.826853
Perplexity: 72.618852
Perplexity: 121.749620
Perplexity: 45.645900
Perplexity: 123.780093
Perplex

In [18]:
from translate.data_utils import get_vocab

In [19]:
VOCAB_DIR = '/Users/PragaashP/Desktop/f16/cs294-129/final-proj/datasets/train/processed'
fr_vocab, fr_rev_vocab = get_vocab(VOCAB_DIR, 50000, lang='fr')

In [20]:
from translate.eval_utils import compute_corpus_bleu_score as bleu_score

In [27]:
def test_model(data, checkpoint_dir):
    """
    Test the translation model.
    """
    def linebreak():
        print('-' * 50)
    
    print("Test neural translation model.")
    linebreak()
    
    graph = tf.Graph()
    
    with graph.as_default():
        with tf.device('/cpu:0'):
            t = time()
            model = Model(source_vocab_size=50000, 
                          target_vocab_size=50000,
                          buckets=_BUCKETS, 
                          size=128,
                          num_layers=3, 
                          batch_size=64,
                          use_lstm=True,
                          optim='adam',
                          num_samples=None, 
                          forward_only=True)
            
            print("Initializing Model took %.6fs" %(time() - t))
            linebreak()
    
    with tf.Session(graph=graph) as sess:
        
        t = time()
        model.load(sess, checkpoint_dir)
        print("Loading Model took %.6fs" %(time() - t))
        linebreak()
        
        t = time()
        score = bleu_score(sess, model, data, _BUCKETS, fr_rev_vocab)
        print("Evaluating BLEU score took %.6fs" %(time() - t))
        linebreak()
        
        print("BLEU Score: %.6f" %(score))

In [28]:
test_model(newstest2013, CKPT_DIR)

Test neural translation model.
--------------------------------------------------
Initializing Model took 27.370849s
--------------------------------------------------
Loading Model took 13.747781s
--------------------------------------------------
Evaluating BLEU score took 193.446001s
--------------------------------------------------
BLEU Score: 0.654400


In [50]:
map(len, newstest2013)

[61, 337, 1038, 1284]